# Ex 1

```sql
CREATE VIEW Summary AS
SELECT
    user_id,
    AVG(number) AS average_number,
    COUNT(id) AS total_responses,
    SUM(CASE WHEN is_correct THEN 1 ELSE 0 END) AS correct_responses
FROM answers
GROUP BY user_id;

SELECT * FROM Summary ;
```

-- version avec calculs par user et exercice
```sql
CREATE VIEW SummaryPerEx AS
SELECT
    user_id,
    exercice_id,
    SUM(number) AS total_number,
    AVG(number) AS average_number,
    COUNT(id) AS total_responses,
    SUM(CASE WHEN is_correct THEN 1 ELSE 0 END) AS correct_responses
FROM answers
GROUP BY user_id, exercice_id;

SELECT * FROM SummaryPerEx ;

```

## 2


### Exemple de calcule de valeur correctes


### Calcule du nombre de correct pour user_id = 1 et exercice_id = 1
```sql
    SELECT SUM(CASE WHEN is_correct THEN 1 ELSE 0 END) as total
    FROM answers
    WHERE answers.user_id = 1 AND answers.exercice_id = 1;
```

### Adaptation en fonction 
**Attention à bien utiliser : `$1` et `$2` lorsque vous variabilisez par rapport à l'exemple précédent**

```sql
CREATE OR REPLACE FUNCTION total_score(user_id INTEGER, exercice_id INTEGER)
RETURNS INTEGER AS $$
DECLARE
    total INTEGER;
BEGIN
    SELECT SUM(CASE WHEN is_correct THEN 1 ELSE 0 END) INTO total
    FROM answers
    WHERE answers.user_id = $1 AND answers.exercice_id = $2;

    RETURN total;
END;
$$ LANGUAGE plpgsql;

```

### Pour score en pourcent (#correct / #total de réponses)

```sql
CREATE OR REPLACE FUNCTION total_score_percentage(user_id INTEGER, exercice_id INTEGER)
RETURNS FLOAT AS $$
DECLARE
    total_correct INTEGER;
    total_answers INTEGER;
BEGIN
    -- #total
    SELECT COUNT(*) INTO total_answers
    FROM answers
    WHERE answers.user_id = $1 AND answers.exercice_id = $2;

    -- Si division par 0 dans le RETURN : il faut prévoir le cas
    IF total_answers = 0 THEN
        RETURN 0;
    END IF;

    -- #correct
    SELECT SUM(CASE WHEN is_correct THEN 1 ELSE 0 END) INTO total_correct
    FROM answers
    WHERE answers.user_id = $1 AND answers.exercice_id = $2;

    -- #correct / #total
    RETURN (total_correct::FLOAT / total_answers) * 100;
END;
$$ LANGUAGE plpgsql;

```

# 3

```sql
CREATE OR REPLACE FUNCTION log_event()
RETURNS TRIGGER AS $$
BEGIN
    -- Insérer un enregistrement dans log_table à chaque nouvel événement
    INSERT INTO log_table (log_message) VALUES ('New event occurred: ' || NEW.event_name);

    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER event_logger
AFTER INSERT ON events
FOR EACH ROW
EXECUTE FUNCTION log_event();

INSERT INTO events (event_name) VALUES ('Event 1');
INSERT INTO events (event_name) VALUES ('Event 2');

SELECT * FROM log_table;

```

